## Asyncio Introduction

- Parallelism means performing multiple operations at the same time
- MultiProcessing means spreading tasks across a computers' central processing unit (cores) to achieve parallelism
- MultiProcessing is fit for CPU-bound tasks (tightly bound loops and mathematical computations fall in this category)
- concurrency means multiple tasks have ability to run in overlapping manner.  It doesn't imply parallelism.  
- threading is concurrent execution model where multiple threads take turns executing tasks.  One process can contain multiple threads
- threading is better for I/O bound tasks (the gil gets released for io bound tasks)
- cpu bound tasks are characterized by computers cores continually workinng hard from start to finish, io bound job is dominated by a lot of waiting time, waiting for an input/output to complete.  
- asyncio is single threaded, single process design which uses cooperative multitasking.  coroutines can be scheduled concurrently


### What does it mean to be asynchronous?

- asynchronous routines are able to pause for a while waiting on their ultimate result and let other routines run in the meantime
- asynchronous code through the above mechanism, facilitates concurrent execution
- Lets look at an example

In [5]:
import asyncio
import time

async def count():
    print("one")
    await asyncio.sleep(1)
    print("two")

async def run():
    s = time.perf_counter()
    await asyncio.gather(count(), count(), count())
    e = time.perf_counter()
    print(f"time elapsed: {(e - s):.2f} seconds")


In [6]:
await run()

one
one
one
two
two
two
time elapsed: 1.00 seconds


- order of execution is at the heart of asycio.  talking to each calls to count() is a single event loop, or coordinator
when each task reaches await asyncio.sleep(1), the function yells up to event loop and gives contol back to it saying "im goin to be sleeping for one second, go ahead and let something else meaninful be done in the meantime"
- contrast that with synchronous version

In [9]:
def count():
    print("one")
    time.sleep(1)
    print("two")

def run():
    for _ in range(3):
        count()

s = time.perf_counter()
run()
e = time.perf_counter()
print(f"time elapsed: {(e - s):.2f} seconds")

one
two
one
two
one
two
time elapsed: 3.02 seconds


- as you can see, benefit of awaiting something with asyncio.sleep() is that surrounding function can temporarily cede control to another function that is more readily able to do something immediately.  
- in contrast, time.sleep() or other blocking call is incompatible with asynchronous python code since it will stop everything in its tracks for the duration of sleep time

## Getting to know Asyncio
- many applications in todays web rely heavily on I/O operations like downloading contents of a web page from internet, running several queries together against a database like PostGres.  A web request or communication with a micro service can take hundreds of milliseconds or seconds if network is slow.  a database query can be time intensive if database is under a high load or query is complex.  A web server may need to handle hundreds or thousands of requests at the same time.  
- making many of these i/o requests at once can be time consuming and lead to substantial performance bottlenecks.  example. if we write an applicatoin to download 100 pages or run 100 queries each of which takes one second to execute, application will take atleast 100 seconds to run.  
- If we use concurrency and start downloads and wait simultaneously, we could complete all of this in under one second
- asyncio was introduced as way to handle these concurrent requests outside of multi processing and multi threading.  Properly using this library can lead to drastic performance and resource utilization improvements for applications that are I/O bound as it allows us to start many of these long running tasks together.  

## What is Asyncio?